In [2]:
import csv
import numpy as np
import timeit
from pathlib import Path
from scipy.stats import spearmanr
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
%matplotlib inline  

In [26]:
file_name='OUTPUT/OUTPUTCoalTimes'
my_demography=np.loadtxt('Examples/example_simple_exp_lin_inc_N_10.txt')
my_samples=np.loadtxt('Examples/example_simple_exp_sample_list.txt') 
my_samples=my_samples.astype(int)
rows=5

In [27]:
mean_coal_times=get_mean_times(file_name, 100)
plot_diversity(my_samples, my_demography, rows, mean_coal_times)


In [36]:
def plot_diversity(my_samples, my_demography,rows, mean_coal_times):
    [T, map_size]=np.shape(my_demography)
    div_map=np.zeros([map_size])
    for i in range (len(my_samples)):
        div_map[(my_samples[i])]=mean_coal_times[i,i]
    div_map_2D=np.reshape(div_map,[rows, int(map_size/rows)])
    plt.pcolor(div_map_2D)
    return(div_map)

def get_mean_times(file_name, n):  #it can take some time
    
    this_data=np.loadtxt(file_name+'1.txt')
    dimension=np.size(this_data,0)
    all_data=np.zeros([dimension,dimension])
    file_count=0
    for i in range(1,n+1):
        this_file_name=(file_name+str(i)+'.txt')
        my_file = Path(this_file_name)
        if my_file.exists():
            file_count=file_count+1
            this_file_data=np.loadtxt(this_file_name)
            all_data=all_data+this_file_data
        else:
            print ('The file', this_file_name, ' does not exists for ', i)
    
    mean_data=all_data/(file_count+0.0)
    np.savetxt(file_name+'MEAN.txt', mean_data)
    return mean_data


In [1]:


def calculate_fst(T):
    Fst=T+1
    return(Fst)